## Imports

In [18]:
import nbimporter
import json
import requests
import time

import undetected_chromedriver as webdriver
from selenium.webdriver.common.by import By

## Start

In [19]:
#startup
driver = webdriver.Chrome()
driver.get("https://www.sportsbet.com.au/")

driver_bf = webdriver.Chrome()
driver_bf.get("https://www.betfair.com.au/exchange/plus/")

## Login using Selenium

In [ ]:
####SPORTSBET
time.sleep(5)
driver.find_element(By.XPATH, "//button[@data-automation-id='header-login-touchable']").click()
time.sleep(3)
driver.find_element(By.XPATH, "//input[@id='username']").send_keys("")
driver.find_element(By.XPATH, "//input[@id='password']").send_keys("")
time.sleep(5)
driver.find_element(By.XPATH,"//div[@data-automation-id='login-button']").click()

####BETFAIR
driver_bf.find_element(By.XPATH, "//input[@id='ssc-liu']").send_keys('')
driver_bf.find_element(By.XPATH, "//input[@id='ssc-lipw']").send_keys("")
time.sleep(5)
driver_bf.find_element(By.XPATH,"//div[@id='ssc-lis']").click()

## Place Bet (Selenium)

In [21]:
def place_bet(horse_id, bf_horse_id, amount):
    driver.find_element(By.XPATH, f'//div[@id="live-stake-{horse_id}"]').send_keys(f"{amount}")
    driver.find_element(By.XPATH, f'//div[@data-automation-id="live-racing-outcome-{horse_id}"]').click()
    driver.find_element(By.XPATH, f'//div[@data-automation-id="live-racing-outcome-{horse_id}"]').click()
    
    driver_bf.find_element(By.XPATH, f'//td[@bet-selection-id="{bf_horse_id}"]').click()
    driver_bf.find_element(By.XPATH, f'//input[@class="betslip-size-input"]').send_keys(f"{amount}")
    driver_bf.find_element(By.XPATH, f'//button[@type="submit"]').click()
    
    print(f"Bet placed: {horse_id, amount}")
    print(f"Bet placed: {bf_horse_id, amount}")
    

## Get Next Race Sportsbet

In [22]:
## GET NEXT RACE
def get_next_race():
    sports_response = requests.get('https://www.sportsbet.com.au/apigw/sportsbook-racing/Sportsbook/Racing/NextEvents?racingFilters=HR_DOMESTIC&groupByFilters=true')
#https://www.sportsbet.com.au/apigw/sportsbook-racing/Sportsbook/Racing/NextEvents?racingFilters=HR_DOMESTIC,HR_INTERNATIONAL,GH_DOMESTIC,GH_INTERNATIONAL,HA_DOMESTIC,HA_INTERNATIONAL&groupByFilters=true
    sports_json = json.loads(sports_response.text)
    #print('Next Race:', json.dumps(sports_json[0], indent =4))   
    sports_json = sports_json[0]
    
    next_race = sports_json["displayName"]
    name = sports_json["competitionName"]
    num =sports_json["raceNumber"]
    sb_id = sports_json["id"]
    
    print('Next Race:', json.dumps(next_race, indent =4))
    
    #selenium get next race
    driver.get(f"https://www.sportsbet.com.au/horse-racing/australia-nz/{name}/race-{num}-{sb_id}")
    #time.sleep(3)
    return sb_id

## Get Next Race Betfair

In [23]:
def bf_get_next_race():
    nextrace_response = requests.get("https://production.ausnav.online/nav?type=screen&name=racing&next2jump=true&today=true")
    nextrace_response = json.loads(nextrace_response.text)
    #print(json.dumps(nextrace_response, indent = 4))
    betfair_response = nextrace_response["nextToJump"]

    betfair_races = []

    for i in betfair_response:
        res = betfair_response[i]
        for j in res:
            r = res[j][0]
            if (r["racingType"] == "Thoroughbred Racing") and ((r["countryCode"] == "AU" or r["countryCode"] == "NZ")) and (r["status"] == "OPEN"):
                betfair_races.append(r)
                #print(json.dumps(r, indent = 4))

    betfair_race_id = betfair_races[0]["marketId"]
    driver_bf.get(f"https://www.betfair.com.au/exchange/plus/horse-racing/market/{betfair_race_id}")
    return betfair_race_id

## Get Prices

In [29]:
def compare_prices(selections, betfair_prices):
    print("SportsBet Prices")
    for idx, i in enumerate(selections):
        #horse details
        print(i["name"])
        horse_id = i["id"]
        print(horse_id)
        bf_horse_id = betfair_prices["selectionId"]
        
        #get prices
        lay_price = betfair_prices[0]["exchange"]["availableToLay"][0]["price"]      
        win_price = i["betPrice"]
        
        #compare prices
        if win_price != "not defined" and i["statistics"]["overview"] != "Scratched.":
            #win price is greater than 105% of the lay
            if win_price >= (lay_price*1.05):
                place_bet(horse_id, bf_horse_id, 5)
            print("Win Price: ", win_price, "\n")

        else:
            print("Scratched \n")


In [30]:
def get_prices_next_race():
    ## Get id of next race
    sb_id = get_next_race()
    bf_id = bf_get_next_race()
    
    
    ## Get prices of next race
    response = requests.get(f"https://www.sportsbet.com.au/apigw/sportsbook-racing/Sportsbook/Racing/Events/{sb_id}/RacecardWithContext?classId=1")
    card_json = json.loads(response.text)
    full_card = card_json.get("racecardEvent")
                #selections = market[0]["selections"]
                #print('Next Race Card:', json.dumps(selections, indent=4))
    selections = full_card["markets"][0]["selections"]

    ### BETFAIR
    prices_response = requests.get(f"https://ero.betfair.com.au/www/sports/exchange/readonly/v1/bymarket?_ak=nzIFcwyWhrlwYMrh&alt=json&currencyCode=AUD&locale=en_GB&marketIds={bf_id}&rollupLimit=25&rollupModel=STAKE&types=MARKET_STATE,RUNNER_STATE,RUNNER_EXCHANGE_PRICES_BEST")
    betfair_prices = json.loads(prices_response.text)
    betfair_prices = betfair_prices["eventTypes"][0]["eventNodes"][0]["marketNodes"][0]["runners"]
    
    
    ### remove scratched 
    for idx, i in enumerate(selections):
        if selections[idx]["statusCode"] == "S":
            selections.remove(selections[idx])

        try:
            selections[idx]["betPrice"] = selections[idx]["prices"][2]["winPrice"]
        except:
            try:
                selections[idx]["betPrice"] = selections[idx]["prices"][0]["winPrice"]
            except:
                 selections[idx]["betPrice"] = "Null"
                
        
        
    return selections, betfair_prices

## Run Bets

In [ ]:
while True:
    selections, betfair_prices = get_prices_next_race()
    print (len(selections))
    print (len(betfair_prices))
    #check current part of race - clock, live, closed, interim
    is_live = driver.find_element(By.XPATH, f'//div[@data-automation-id="racecard-clock"]//span').text

    if is_live == "LIVE" and (betfair_prices[i]["status"] == "ACTIVE"):
        while is_live == "LIVE" and (betfair_prices[i]["status"] == "ACTIVE"):
            if len(selections) == len(betfair_prices):
                compare_prices(selections, betfair_prices)
                time.sleep(3)

    else:
        print(is_live)


## Get Account Details